<a href="https://colab.research.google.com/github/Feelingorcas/Kaggle_study/blob/master/Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## code for kaggle  https://www.kaggle.com/competitions/titanic/data
## first introduction for kaggle 



## 데이터 받고, 전 처리 하고, 모델학습 시키고, 결과 얻기.




In [ ]:
## librarys should include hear to get up
from google.colab import drive
drive.mount("/content/MyDrive/") 
import torch 
import torch.nn 
import pandas as pd 
import numpy as np 

Mounted at /content/MyDrive/


In [ ]:
## read csv from local directory 

train_directory = '/content/MyDrive/MyDrive/Kaggles/kaggle_titanic/train.csv' 
test_directory = '/content/MyDrive/MyDrive/Kaggles/kaggle_titanic/test.csv'
train_data = pd.read_csv(train_directory) 
test_data = pd.read_csv(test_directory)


# Sex 랑 Embarked, Name

train_data_frame =  pd.DataFrame(train_data,columns = ['Survived','Name','Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']) 
test_data_frame = pd.DataFrame(test_data,columns =['Name','Pclass','Sex','Age','SibSp','Parch','Fare','Embarked'])

merged = (train_data_frame,test_data_frame )
##개략적인 통계자료 보여주는 것
#dataset.describe() 

# title 이라는 것으로 mr mrs 등 sex 관련하여 새로운 범주 데이터 생성 
for dataset in merged :
      dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
      dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
 	'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

for dataset in merged : 
     dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
     dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
     dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
     dataset = dataset.drop("Name",axis=1)

#print(dataset)

## 각 범주별 생존률을 따로 보여주는 것
print(dataset[["Sex", "Survived"]].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False))
#print(dataset[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False))
#print( dataset[['Embarked','Survived']].groupby(['Embarked'],as_index=False).mean().sort_values(by='Survived',ascending=False))
#print( dataset[['Title','Survived']].groupby(['Title'],as_index=False).mean().sort_values(by='Survived',ascending=False))

# 범주형 데이터 숫자로 변환 

for dataset in merged :
      dataset['Sex'] = dataset['Sex'].map(({'male': 1 , 'female':0})).astype(int)
      dataset['Embarked'] = dataset['Embarked'].fillna('S')
      dataset['Title'] = dataset['Title'].map({'Mrs':5,'Miss':4,'Master':3,'Rare':2,'Mr':1}).astype(int)
      dataset['Embarked'] = dataset['Embarked'].map(({'Q':3 ,'S':2 ,'C':1})).astype(int)

 
## age nan 성별를 분류해서 바꾸기
for dataset in merged :
      dataset['Age'].fillna(dataset.groupby('Pclass')['Age'].transform('mean'), inplace=True)
      dataset['Age'].fillna(dataset.groupby('Pclass')['Age'].transform('mean'), inplace=True)
      dataset['Fare'].fillna(dataset.groupby('Pclass')['Fare'].transform('mean'),inplace=True)
      

## Embarked 범주형 데이터 더미데이터로 바꾸기
#dataset = pd.get_dummies(dataset)
#dataset = pd.get_dummies(dataset)

train_data_frame['AgeBand'] = pd.qcut(train_data_frame['Age'], 5)
print(train_data_frame[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True))
##연속형 변수인 age 랑 Fare 를 범주형 변수로 바꿔주기
for dataset in merged :  
     dataset['FareBand'] = pd.qcut(dataset['Fare'], 6)
     #print(dataset[['FareBand', 'Survived']].groupby(['FareBand'], as_index=False).mean().sort_values(by='FareBand', ascending=True))
     
     dataset.loc[(dataset['Fare']<=7.775),'Fare'] = 0
     dataset.loc[(8.662>=dataset['Fare'])&(dataset['Fare']>7.775) ,'Fare']=1
     dataset.loc[(8.662<dataset['Fare'])&(dataset['Fare']<14.454),'Fare']=2
     dataset.loc[(14.454<=dataset['Fare'])&(dataset['Fare']<26.0),'Fare']=3
     dataset.loc[(52.369>=dataset['Fare'])&(dataset['Fare']>=26.0),'Fare']=4
     dataset.loc[(52.369<dataset['Fare']),'Fare']=5
     
     dataset['Fare']= dataset['Fare'].astype(int)

     dataset.loc[(0.419<dataset['Age']) & (dataset['Age']<=20.0),'Age'] = 0
     dataset.loc[(20.0<dataset['Age']) & (dataset['Age']<=25.141),'Age'] = 1
     dataset.loc[(25.141<dataset['Age']) & (dataset['Age']<=30),'Age'] = 2
     dataset.loc[(30.0<dataset['Age']) & (dataset['Age']<=38.233),'Age'] = 3
     dataset.loc[(38.233<dataset['Age']) & (dataset['Age']<=80),'Age'] = 4
     dataset['Age']= dataset['Age'].astype(int)


print(test_data_frame.drop(['Name'],axis= 1))


          AgeBand  Survived
0   (0.419, 20.0]  0.458101
1  (20.0, 25.141]  0.294574
2  (25.141, 30.0]  0.386555
3  (30.0, 38.233]  0.455696
4  (38.233, 80.0]  0.372881
     Pclass  Sex  Age  SibSp  Parch  Fare  Embarked  Title           FareBand
0         3    1    3      0      0     1         3      1     (7.762, 8.662]
1         3    0    4      1      0     0         2      5    (-0.001, 7.762]
2         2    1    4      0      0     2         3      1    (8.662, 14.454]
3         3    1    2      0      0     2         2      1     (7.762, 8.662]
4         3    0    1      1      1     2         2      5    (8.662, 14.454]
..      ...  ...  ...    ...    ...   ...       ...    ...                ...
413       3    1    1      0      0     1         2      1     (7.762, 8.662]
414       1    0    4      0      0     5         1      2  (55.969, 512.329]
415       3    1    4      0      0     0         2      1    (-0.001, 7.762]
416       3    1    1      0      0     1         2 

In [ ]:
X_train = train_data_frame.drop(['Survived','Name','AgeBand','FareBand'],axis=1).to_numpy()
Y_train = train_data_frame['Survived'].to_numpy()
X_test = test_data_frame.drop(['Name','FareBand'],axis=1).to_numpy()
print(X_train.shape)
print(Y_train.shape)

X_train = torch.from_numpy(X_train)
Y_train = torch.from_numpy(Y_train)


print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)


(891, 8)
(891,)
torch.Size([891, 8])
torch.Size([891])
(418, 8)


In [ ]:
# model 수요일에 끝내기
import torch.nn as nn 

class MyModel(nn.Module) :
    def __init__(self) :
      super(MyModel, self).__init__()
      self.fc1 = nn.Linear(8,1024)
      self.relu1 = nn.ReLU() 
      self.fc2 = nn.Linear(1024,1024)
      self.relu2 = nn.ReLU()
      self.fc3 = nn.Linear(1024,2)
      
    
    def forward(self, x) :
       x = self.relu1(self.fc1(x))
       x = self.relu2(self.fc2(x))
       x = self.fc3(x)
     
       return x 
  

   



In [ ]:
#Batch가 필요한 데이터는 아니니.
learning_rate = 0.1 
model = MyModel() ; 
optimizer = torch.optim.SGD(params=model.parameters(),lr=learning_rate,momentum=0.9)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer,gamma=0.9) 
loss = torch.nn.CrossEntropyLoss()




In [ ]:
#training 

epoch = 100

for i in range(epoch) :
   loss.zero_grad() 
   y_pre  = model(X_train)
   loss = loss(Y_train,y_pre)
   print("loss :"+loss)
   loss.backward()
   optimizer.step()
   scheduler.step()






RuntimeError: ignored